In [ ]:
# Librerías estándar de Python
import os  # Proporciona funciones para interactuar con el sistema operativo (por ejemplo, manejo de archivos y directorios).
import pickle  # Permite la serialización y deserialización de objetos de Python (guardar y cargar estructuras de datos).

# Librerías de terceros para análisis y visualización de datos
import pandas as pd  # Herramientas para manipulación y análisis de datos estructurados (DataFrames, series temporales).
# Importación de funciones propias del proyecto
from preprocessing import extract_features_with_majority_label, parse_personal_info, extract_float
import re
import numpy as np
from collections import Counter
import time

In [7]:
# Definición de la ruta al directorio WESAD y carga del archivo S11.pkl
WESAD_ROOT = "/Users/gerardoaboulafia/Library/Mobile Documents/com~apple~CloudDocs/UCA/Documentos/Cuatrimestre 5/Laboratorio/WESAD"

with open(os.path.join(WESAD_ROOT, "S11", "S11.pkl"), "rb") as f:
    data = pickle.load(f, encoding='latin1')

# Exploramos las claves principales
print("Claves del diccionario principal:", data.keys())
print("Claves de 'signal':", data['signal'].keys())

Claves del diccionario principal: dict_keys(['signal', 'label', 'subject'])
Claves de 'signal': dict_keys(['chest', 'wrist'])


-----

En esta segunda sección se implementa un pipeline completo de procesamiento y análisis del dataset WESAD para múltiples pacientes. El objetivo es extraer información fisiológica relevante, vincularla con metadatos personales, y entrenar un modelo de clasificación supervisado para distinguir entre dos estados emocionales (etiquetas 1 y 2).

Para cada paciente:
- Se busca y carga automáticamente el archivo .pkl con las señales fisiológicas y el archivo readme.txt con la información demográfica.
- Se aplica una función de extracción de características basada en ventanas deslizantes sobre las señales ACC, EDA y Temperatura, generando un vector de 17 estadísticas por ventana (media, desvío estándar, máximos, pendiente, etc.).
- Se asigna la etiqueta mayoritaria de cada ventana y se estructura todo en un DataFrame enriquecido con los datos del participante (edad, género, mano dominante, etc.).

Posteriormente, se concatenan los datos de todos los sujetos en un único DataFrame global.

In [8]:
#WESAD_ROOT = "/Users/gerardoaboulafia/Library/Mobile Documents/com~apple~CloudDocs/UCA/Documentos/Cuatrimestre 5/Laboratorio/WESAD"
WESAD_ROOT = "/Users/gerardoaboulafia/Library/Mobile Documents/com~apple~CloudDocs/UCA/Documentos/Cuatrimestre 5/Laboratorio/WESAD"

# ------------------------------------------------------------------
# Procesamiento de todos los sujetos
# ------------------------------------------------------------------

all_subjects = []

for subject_dir in sorted(os.listdir(WESAD_ROOT)):
    subject_path = os.path.join(WESAD_ROOT, subject_dir)
    if not os.path.isdir(subject_path):
        continue

    # Busca el .pkl
    pkl_path = None
    for root, _, files in os.walk(subject_path):
        for file in files:
            if file.lower().endswith(".pkl"):
                pkl_path = os.path.join(root, file)
                break
        if pkl_path:
            break
    if not pkl_path:
        print(f" No se encontró .pkl en {subject_dir}")
        continue

    # Busca el readme.txt con info demográfica
    txt_path = None
    for root, _, files in os.walk(subject_path):
        for file in files:
            if file.lower().endswith(".txt") and "readme" in file.lower():
                txt_path = os.path.join(root, file)
                break
        if txt_path:
            break

    # Carga el archivo
    try:
        with open(pkl_path, "rb") as f:
            data = pickle.load(f, encoding='latin1')
    except Exception as e:
        print(f" Error cargando {subject_dir}: {e}")
        continue

    print(f" Procesando {subject_dir}...")
    X, y = extract_features_with_majority_label(data, verbose=True)

    # Lo pasamos a DataFrame
    colnames = [
        'acc_mean', 'acc_std', 'acc_maxx', 'acc_maxy', 'acc_maxz',
        'eda_max', 'eda_min', 'eda_mean', 'eda_range', 'eda_std', 'eda_slope',
        'tmp_max', 'tmp_min', 'tmp_mean', 'tmp_range', 'tmp_std'
    ]
    df = pd.DataFrame(X, columns=colnames)
    df["Label"] = y
    df["Paciente"] = subject_dir

    if txt_path:
        info = parse_personal_info(txt_path)
        for k, v in info.items():
            df[k] = v

    all_subjects.append(df)
    print(f" {subject_dir} procesado: {df.shape[0]} ventanas")

# ------------------------------------------------------------------
# Unión final
# ------------------------------------------------------------------

if not all_subjects:
    raise RuntimeError("No se procesó ningún sujeto.")

df_final = pd.concat(all_subjects, ignore_index=True)
print(" DataFrame final:", df_final.shape)

 Procesando S10...
Procesado: 0/21744 ventanas (0.0%)
Procesado: 1088/21744 ventanas (5.0%)
Procesado: 2176/21744 ventanas (10.0%)
Procesado: 3264/21744 ventanas (15.0%)
Procesado: 4352/21744 ventanas (20.0%)
Procesado: 5440/21744 ventanas (25.0%)
Procesado: 6528/21744 ventanas (30.0%)
Procesado: 7616/21744 ventanas (35.0%)
Procesado: 8704/21744 ventanas (40.0%)
Procesado: 9792/21744 ventanas (45.0%)
Procesado: 10880/21744 ventanas (50.0%)
Procesado: 11968/21744 ventanas (55.0%)
Procesado: 13056/21744 ventanas (60.0%)
Procesado: 14144/21744 ventanas (65.0%)
Procesado: 15232/21744 ventanas (70.1%)
Procesado: 16320/21744 ventanas (75.1%)
Procesado: 17408/21744 ventanas (80.1%)
Procesado: 18496/21744 ventanas (85.1%)
Procesado: 19584/21744 ventanas (90.1%)
Procesado: 20672/21744 ventanas (95.1%)

 Extracción finalizada en 229.72 segundos.
 S10 procesado: 21744 ventanas


/Users/gerardoaboulafia/Downloads/preprocessing.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(features_list), np.array(labels_list)


 Procesando S11...
Procesado: 0/20692 ventanas (0.0%)
Procesado: 1035/20692 ventanas (5.0%)
Procesado: 2070/20692 ventanas (10.0%)
Procesado: 3105/20692 ventanas (15.0%)
Procesado: 4140/20692 ventanas (20.0%)
Procesado: 5175/20692 ventanas (25.0%)
Procesado: 6210/20692 ventanas (30.0%)
Procesado: 7245/20692 ventanas (35.0%)
Procesado: 8280/20692 ventanas (40.0%)
Procesado: 9315/20692 ventanas (45.0%)
Procesado: 10350/20692 ventanas (50.0%)
Procesado: 11385/20692 ventanas (55.0%)
Procesado: 12420/20692 ventanas (60.0%)
Procesado: 13455/20692 ventanas (65.0%)
Procesado: 14490/20692 ventanas (70.0%)
Procesado: 15525/20692 ventanas (75.0%)
Procesado: 16560/20692 ventanas (80.0%)
Procesado: 17595/20692 ventanas (85.0%)
Procesado: 18630/20692 ventanas (90.0%)
Procesado: 19665/20692 ventanas (95.0%)

 Extracción finalizada en 216.16 segundos.
 S11 procesado: 20692 ventanas


/Users/gerardoaboulafia/Downloads/preprocessing.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(features_list), np.array(labels_list)


 Procesando S13...
Procesado: 0/21908 ventanas (0.0%)
Procesado: 1096/21908 ventanas (5.0%)
Procesado: 2192/21908 ventanas (10.0%)
Procesado: 3288/21908 ventanas (15.0%)
Procesado: 4384/21908 ventanas (20.0%)
Procesado: 5480/21908 ventanas (25.0%)
Procesado: 6576/21908 ventanas (30.0%)
Procesado: 7672/21908 ventanas (35.0%)
Procesado: 8768/21908 ventanas (40.0%)
Procesado: 9864/21908 ventanas (45.0%)
Procesado: 10960/21908 ventanas (50.0%)
Procesado: 12056/21908 ventanas (55.0%)
Procesado: 13152/21908 ventanas (60.0%)
Procesado: 14248/21908 ventanas (65.0%)
Procesado: 15344/21908 ventanas (70.0%)
Procesado: 16440/21908 ventanas (75.0%)
Procesado: 17536/21908 ventanas (80.0%)
Procesado: 18632/21908 ventanas (85.0%)
Procesado: 19728/21908 ventanas (90.0%)
Procesado: 20824/21908 ventanas (95.1%)

 Extracción finalizada en 226.41 segundos.
 S13 procesado: 21908 ventanas


...

 Procesando S8...
Procesado: 0/21624 ventanas (0.0%)
Procesado: 1082/21624 ventanas (5.0%)
Procesado: 2164/21624 ventanas (10.0%)
Procesado: 3246/21624 ventanas (15.0%)
Procesado: 4328/21624 ventanas (20.0%)
Procesado: 5410/21624 ventanas (25.0%)
Procesado: 6492/21624 ventanas (30.0%)
Procesado: 7574/21624 ventanas (35.0%)
Procesado: 8656/21624 ventanas (40.0%)
Procesado: 9738/21624 ventanas (45.0%)
Procesado: 10820/21624 ventanas (50.0%)
Procesado: 11902/21624 ventanas (55.0%)
Procesado: 12984/21624 ventanas (60.0%)
Procesado: 14066/21624 ventanas (65.0%)
Procesado: 15148/21624 ventanas (70.1%)
Procesado: 16230/21624 ventanas (75.1%)
Procesado: 17312/21624 ventanas (80.1%)
Procesado: 18394/21624 ventanas (85.1%)
Procesado: 19476/21624 ventanas (90.1%)
Procesado: 20558/21624 ventanas (95.1%)

 Extracción finalizada en 220.50 segundos.
 S8 procesado: 21624 ventanas


/Users/gerardoaboulafia/Downloads/preprocessing.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(features_list), np.array(labels_list)


 Procesando S9...
Procesado: 0/20652 ventanas (0.0%)
Procesado: 1033/20652 ventanas (5.0%)
Procesado: 2066/20652 ventanas (10.0%)
Procesado: 3099/20652 ventanas (15.0%)
Procesado: 4132/20652 ventanas (20.0%)
Procesado: 5165/20652 ventanas (25.0%)
Procesado: 6198/20652 ventanas (30.0%)
Procesado: 7231/20652 ventanas (35.0%)
Procesado: 8264/20652 ventanas (40.0%)
Procesado: 9297/20652 ventanas (45.0%)
Procesado: 10330/20652 ventanas (50.0%)
Procesado: 11363/20652 ventanas (55.0%)
Procesado: 12396/20652 ventanas (60.0%)
Procesado: 13429/20652 ventanas (65.0%)
Procesado: 14462/20652 ventanas (70.0%)
Procesado: 15495/20652 ventanas (75.0%)
Procesado: 16528/20652 ventanas (80.0%)
Procesado: 17561/20652 ventanas (85.0%)
Procesado: 18594/20652 ventanas (90.0%)
Procesado: 19627/20652 ventanas (95.0%)

 Extracción finalizada en 210.81 segundos.
 S9 procesado: 20652 ventanas


/Users/gerardoaboulafia/Downloads/preprocessing.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(features_list), np.array(labels_list)


 DataFrame final: (343872, 23)
